In [1]:
import pandas as pd
import os

In [2]:
def read_in_dfs(tables):
    df_list = []
    for table in tables:
        globals()[table.split('.')[0]] = pd.read_csv(f"../clean_datasets/{table}")
        df_list.append(globals()[table.split('.')[0]])
        print(f'{table.split(".")[0]} is done')

In [3]:
exempt = ['clean_ridership', 'clean_ridership_totals', 'clean_divvy_trips', 'clean_disadvantaged_areas', 'clean_311', 'clean_crime']

In [4]:
def gather_files(directory):
    tables = []
    for foldername, subfolders, filenames in os.walk(directory):
        for filename in filenames:
            if filename.split('.')[0] not in exempt:
                tables.append(filename)
                print(f'{filename} successfully identified')
    return tables

In [31]:
root_directory = '../clean_datasets/'
read_in_dfs(gather_files(root_directory))

clean_cta_stations.csv successfully identified
clean_police_stations.csv successfully identified
clean_bus_stations.csv successfully identified
clean_bike_stations.csv successfully identified
clean_areas.csv successfully identified
clean_public_healthindicator.csv successfully identified
clean_cta_stations is done
clean_police_stations is done
clean_bus_stations is done
clean_bike_stations is done
clean_areas is done
clean_public_healthindicator is done


In [32]:
areas = clean_areas[['id']]
areas.sort_values(by='id', inplace=True)
areas.reset_index(drop=True, inplace=True)
areas

/var/folders/5d/fdmfhfr55d5f2r62ltcl50vh0000gn/T/ipykernel_17815/3352781940.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  areas.sort_values(by='id', inplace=True)


,id
0,1
1,2
2,3
3,4
4,5
...,...
72,73
73,74
74,75
75,76


In [33]:
clean_cta_stations

,station_number,station_name,lat,long,status,area
0,1,18th (Pink Line),41.857908,-87.669147,False,31
1,2,35th-Bronzeville-IIT (Green Line),41.831677,-87.625826,False,35
2,3,35th/Archer (Orange Line),41.829353,-87.680622,False,59
3,4,43rd (Green Line),41.816462,-87.619021,True,38
4,5,47th (Green Line),41.809209,-87.618826,True,38
...,...,...,...,...,...,...
138,139,Western (Blue Line - O'Hare Branch),41.916157,-87.687364,False,22
139,140,Western (Brown Line),41.966163,-87.688502,False,4
140,141,Western (Orange Line),41.804546,-87.684019,True,63
141,142,Western (Pink Line),41.854225,-87.685129,False,31


In [34]:
clean_cta_stations.area.value_counts()

area
0     20
32    16
28     9
8      7
6      7
25     5
31     4
1      4
38     4
4      4
24     4
29     4
27     4
77     4
14     3
22     3
3      3
16     3
7      2
33     2
26     2
37     2
68     2
76     2
44     2
42     2
34     2
5      2
10     1
62     1
21     1
56     1
58     1
11     1
59     1
60     1
49     1
40     1
35     1
30     1
67     1
69     1
63     1
Name: count, dtype: int64

In [35]:
grouped_cta = clean_cta_stations[clean_cta_stations.area != 0].groupby('area')['station_number'].agg('count').reset_index().sort_values(by='station_number', ascending=False).reset_index(drop=True)
areas = areas.merge(grouped_cta, left_on='id', right_on='area', how='outer').fillna(0)
areas.drop(columns=['area'], inplace=True)
areas.rename(columns={'station_number' : 'cta_stations'}, inplace=True)
areas

,id,cta_stations
0,1,4.0
1,2,0.0
2,3,3.0
3,4,4.0
4,5,2.0
...,...,...
72,73,0.0
73,74,0.0
74,75,0.0
75,76,2.0


In [36]:
clean_police_stations.head()

,id,lat,long,status,area
0,0,41.830702,-87.623395,True,35
1,18,41.903242,-87.643352,False,8
2,19,41.947400,-87.651512,False,6
3,20,41.979550,-87.692845,False,4
4,22,41.691435,-87.668520,False,75


In [37]:
grouped_police = clean_police_stations.groupby('area')['lat'].agg('count').reset_index().sort_values(by='lat', ascending=False).reset_index(drop=True)
areas = areas.merge(grouped_police, left_on='id', right_on='area', how='left').fillna(0)
areas.drop(columns=['area'], inplace=True)
areas.rename(columns={'lat' : 'police_stations'}, inplace=True)
areas

,id,cta_stations,police_stations
0,1,4.0,1.0
1,2,0.0,0.0
2,3,3.0,0.0
3,4,4.0,1.0
4,5,2.0,0.0
...,...,...,...
72,73,0.0,0.0
73,74,0.0,0.0
74,75,0.0,1.0
75,76,2.0,0.0


In [38]:
clean_bus_stations

,stop_id,cta_stop_name,lat,long,status,area
0,68,Jackson & Franklin,41.878051,-87.635401,False,32
1,69,Jackson & Financial Place,41.878081,-87.632868,False,32
2,73,Michigan & Van Buren/Congress,41.876271,-87.624396,False,32
3,75,Michigan & Van Buren,41.876690,-87.624160,False,32
4,76,Michigan & Jackson,41.878135,-87.624193,False,32
...,...,...,...,...,...,...
393,17706,100th Street & Paxton,41.713395,-87.569633,False,51
394,17967,Cottage Grove & 67th Street,41.772906,-87.605894,True,42
395,17379,Belmont Red/Brown/Purple Line Station,41.939870,-87.653062,False,6
396,14488,Michigan & Superior,41.895823,-87.624422,False,8


In [39]:
grouped_bus = clean_bus_stations.groupby('area')['stop_id'].agg('count').reset_index().sort_values(by='stop_id', ascending=False).reset_index(drop=True)
areas = areas.merge(grouped_bus, left_on='id', right_on='area', how='left').fillna(0)
areas.drop(columns=['area'], inplace=True)
areas.rename(columns={'stop_id' : 'bus_stations'}, inplace=True)
areas

,id,cta_stations,police_stations,bus_stations
0,1,4.0,1.0,7.0
1,2,0.0,0.0,2.0
2,3,3.0,0.0,3.0
3,4,4.0,1.0,5.0
4,5,2.0,0.0,5.0
...,...,...,...,...
72,73,0.0,0.0,2.0
73,74,0.0,0.0,2.0
74,75,0.0,1.0,1.0
75,76,2.0,0.0,1.0


In [40]:
clean_public_healthindicator

,id,unemployment,per_capita_income,no_hs_dip,gov_depend,crowded_housing,below_pov
0,1,0.075,23714,0.181,0.288,0.079,0.227
1,2,0.079,21375,0.196,0.383,0.070,0.151
2,3,0.077,32355,0.136,0.222,0.046,0.227
3,4,0.068,35503,0.125,0.256,0.031,0.095
4,5,0.045,51615,0.054,0.255,0.002,0.071
...,...,...,...,...,...,...,...
72,73,0.183,19709,0.156,0.424,0.011,0.157
73,74,0.069,34221,0.045,0.370,0.011,0.031
74,75,0.149,26185,0.109,0.394,0.008,0.137
75,76,0.047,29402,0.110,0.265,0.019,0.095


In [41]:
areas = areas.merge(clean_public_healthindicator, on='id', how='left')
areas

,id,cta_stations,police_stations,bus_stations,unemployment,per_capita_income,no_hs_dip,gov_depend,crowded_housing,below_pov
0,1,4.0,1.0,7.0,0.075,23714,0.181,0.288,0.079,0.227
1,2,0.0,0.0,2.0,0.079,21375,0.196,0.383,0.070,0.151
2,3,3.0,0.0,3.0,0.077,32355,0.136,0.222,0.046,0.227
3,4,4.0,1.0,5.0,0.068,35503,0.125,0.256,0.031,0.095
4,5,2.0,0.0,5.0,0.045,51615,0.054,0.255,0.002,0.071
...,...,...,...,...,...,...,...,...,...,...
72,73,0.0,0.0,2.0,0.183,19709,0.156,0.424,0.011,0.157
73,74,0.0,0.0,2.0,0.069,34221,0.045,0.370,0.011,0.031
74,75,0.0,1.0,1.0,0.149,26185,0.109,0.394,0.008,0.137
75,76,2.0,0.0,1.0,0.047,29402,0.110,0.265,0.019,0.095


In [42]:
clean_bike_stations = pd.read_csv('../clean_datasets/clean_bike_stations.csv')
clean_bike_stations

,id,station_name,lat,long,status,area
0,1594046383808271024,Troy St & Jackson Blvd,41.877505,-87.704850,1,27
1,641,Central Park Ave & Bloomingdale Ave,41.914166,-87.716755,1,22
2,1683527931525155814,Public Rack - Cornell Ave & 87th Pl,41.736881,-87.583146,0,48
3,367,Racine Ave & 35th St,41.830689,-87.656211,0,60
4,1673852313397164648,Public Rack - Francisco Ave & Touhy Ave,42.011865,-87.701317,0,2
...,...,...,...,...,...,...
1411,20,Sheffield Ave & Kingsbury St,41.910522,-87.653106,0,8
1412,54,Ogden Ave & Chicago Ave,41.896362,-87.654061,0,24
1413,693,Baltimore Ave & 87th St,41.737336,-87.548515,1,46
1414,718,Ewing Ave & Burnham Greenway,41.712749,-87.534814,1,52


In [43]:
grouped_bike = clean_bike_stations.groupby('area')['station_name'].agg('count').reset_index().sort_values(by='station_name', ascending=False).reset_index(drop=True)
areas = areas.merge(grouped_bike, left_on='id', right_on='area', how='left').fillna(0)
areas.drop(columns=['area'], inplace=True)
areas.rename(columns={'station_name' : 'bike_stations'}, inplace=True)
areas

,id,cta_stations,police_stations,bus_stations,unemployment,per_capita_income,no_hs_dip,gov_depend,crowded_housing,below_pov,bike_stations
0,1,4.0,1.0,7.0,0.075,23714,0.181,0.288,0.079,0.227,17.0
1,2,0.0,0.0,2.0,0.079,21375,0.196,0.383,0.070,0.151,30.0
2,3,3.0,0.0,3.0,0.077,32355,0.136,0.222,0.046,0.227,15.0
3,4,4.0,1.0,5.0,0.068,35503,0.125,0.256,0.031,0.095,19.0
4,5,2.0,0.0,5.0,0.045,51615,0.054,0.255,0.002,0.071,15.0
...,...,...,...,...,...,...,...,...,...,...,...
72,73,0.0,0.0,2.0,0.183,19709,0.156,0.424,0.011,0.157,23.0
73,74,0.0,0.0,2.0,0.069,34221,0.045,0.370,0.011,0.031,16.0
74,75,0.0,1.0,1.0,0.149,26185,0.109,0.394,0.008,0.137,25.0
75,76,2.0,0.0,1.0,0.047,29402,0.110,0.265,0.019,0.095,0.0


In [52]:
areas.to_csv('../scoring_datasets/area_reference.csv', index=False)